# Защита персональных данных клиентов

Нам необходимо разработать алгоритм преобразования данных, который максимально усложнит процедуру восстановления исходных, при этом не ухудшит качество моделей машинного обучения. 

На входе у нас 5000 строк данных клиентов компании. 

План работ: 
1. Загрузим и предобработаем данные.
2. Оценим качество модели машинного обучения без обезличивания данных.
3. Проверим гипотезу о том, что качество модели машинного обучения не изменится при умножении признаков на обратимую матрицу.
4. Составим и проверим алгоритм шифрования данных.

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
df = pd.read_csv('/datasets/insurance.csv')
df.columns = ['sex', 'age', 'salary', 'family_members', 'payouts']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   sex             5000 non-null   int64  
 1   age             5000 non-null   float64
 2   salary          5000 non-null   float64
 3   family_members  5000 non-null   int64  
 4   payouts         5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,sex,age,salary,family_members,payouts
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Данные выглядят корректно, предобработка не требуется. 

Разделим выборку на обучающую и валдиационную. Также, разделим признаки на целевые и обучающие.

In [5]:
features = df.drop(columns = ['payouts'])
target = df['payouts']

features_train, features_valid, target_train, target_valid = train_test_split(features, target, random_state=12345, test_size = 0.25)

In [6]:
print (features_train.shape)
print (features_valid.shape)
print (target_train.shape)
print (target_valid.shape)

(3750, 4)
(1250, 4)
(3750,)
(1250,)


Обучающие и валидационные наборы данных созданы. Можем приступать к обучению моделей. 

## Умножение матриц


$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Умножим матрицу признаков X на обратимую матрицу P:

$$
a' = (XP)w'
$$


$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

$$
w' = (P^T X^T X P)^{-1} (XP)^T y
$$

По свойству умножения матриц, внутри выражения возможно расставлять скобки любым образом, если не меняется порядок матриц: 
$$
A (B C) = (A B) C
$$
Запишем выражение $w$ следующим образом: 

$$
w' = (P^T ((X^T X) P))^{-1} (XP)^T y
$$

Раскроем скобки по свойству умножения матриц, тогда: 
$$
w' = ((X^T X) P)^{-1} (P^T)^{-1} (XP)^T y
$$
=>
$$
w' = ((X^T X) P)^{-1} (P^T)^{-1} P^T X^T y
$$


$$
(P^T)^{-1} P^T = E
$$
=>
$$
w' = ((X^T X) P)^{-1} X^T y
$$
=>
$$
w' = P^{-1} (X^T X)^{-1} X^T y
$$

Запишем выражение $a'$:

$$
a' = (X P) P^{-1} (X^T X)^{-1} X^T y
$$

$$ P P^{-1} = E $$
=>

$$
a' = X (X^T X)^{-1} X^T y
$$
Доказано, что a' = a
        
</p>     
</div>

Создадим модель машинного обучения, методом logisticRegression. Для оценки качества обученной модели будем использовать метрику r2

In [7]:
model = LinearRegression()
model.fit(features_train, target_train)
results = model.predict(features_valid)
r2_score(target_valid, results)

0.43522757127025635

Создадим обратимую матрицу для умножения признаков. 
Эта матрица должна быть квадратной, так как только квадратные матрицы обратимы, и иметь высоту = 4, так как длина матрицы признаков = 4. 

Итого, создаем матрицу 4х4. 

Далее, обучим модель в цикле 100 раз, оценивая ее качество метрикой r2, затем найдем среднее значение и его отношение к значению r2 метрики, полученному при обучении на исходных признаках. 

In [8]:
def is_matrix_inv (matrix):
    try:
        np.linalg.inv(matrix)
        return True
    except:
        return False

In [9]:
scores = []
for i in range (1,100):
    matrix = np.random.randint(10,size = (4,4))
    while not is_matrix_inv(matrix):
        matrix = np.random.randint(10,size = (4,4))
    new_features_train = features_train.values@matrix
    new_features_valid = features_valid.values@matrix
    model = LinearRegression()
    model.fit(new_features_train, target_train)
    results = model.predict(new_features_valid)
    scores.append (r2_score(target_valid, results))
(np.array(scores).mean()/r2_score(target_valid, results))*100

100.0000000000214

Точность выше 99% на матрице, состоящей из положительных целых чисел одного порядка. 
Попробуем составить более неоднородную матрицу, где значения отличабются друг от друга на несколько порядков. 

In [10]:
a = np.array([[10000000000000000000000000000.0,-0.1,0,-0.23],[1,-0.1,-1,0],[0,100,-0.1,1],[0,0,10,1000]])
if is_matrix_inv(a):
    new_features_train = features_train.values@a
    new_features_valid = features_valid.values@a
    model = LinearRegression()
    model.fit(new_features_train, target_train)
    results = model.predict(new_features_valid)
    print (r2_score(target_valid, results))
else:
    print ('матрица необратима')

-0.0001430237930319933


Значение метрики пошло вниз. Это связано с тем, что часть признаков умножается на слишком большое число, часть на слишком малое, что создает дисбаланс классов и, как следствие, нерелевантный вектор w, который бесполезен для данных, отличных от обучающих. 

Если эта гипотеза верна, то нам поможет скалирование признаков. Выполним его, далее переобучим модель и найдем метрику r2.  

In [11]:
scaler = StandardScaler()
scaler.fit(new_features_train)
new_features_train = scaler.transform(new_features_train)
new_features_valid = scaler.transform(new_features_valid)
model = LinearRegression()
model.fit(new_features_train, target_train)
results = model.predict(new_features_valid)
r2_score(target_valid, results)

0.43522757127026657

Значение метрики восстановлено! 

Возьмем выполненные операции за основу для алгоритма обезличивания данных. 

## Алгоритм преобразования

Наш алгоритм будет принимать таблицу признаков на вход, и возвращать обезличенные признаки также одной таблицей. 
Работа алгоритма будет состоять из следующих шагов: 
1. Генерация квадратной обратимой матрицы, шириной, равной ширине таблицы признаков и проверка матрицы на обратимость.
2. Умножение исходной таблицы на матрицу
3. Скалирование признаков в получившейся матрице признаков. 
4. Возвращение матрицы признаков и сгенерированной матрицы. 

Далее, преобразованные признаки можно передать дата-саентистам для обработки, матрицу сохранить для последующего умножения боевых данных, чтобы обученная модель дала правильные результаты.

Для проверки алгоритма обучим модель на имеющихся данных, далее сравним значение метрики r2 валидационной выборки с метрикой r2 после обучения на данных, умноженных на матрицу.

Плюсы подхода: 
1. Уникальная матрица каждую итерацию обработки данных, это защищает нас от риска восстановления всех данных в случае утечки матрицы. 

Минусы: 
1. Необходимо развернуть и аккуратно вести базу данных матриц со связкой 1:1 к каждой обученной модели, а также нужен сервис, который умножит продовые данные на матрицу. Так как проект не подразумевает базу данных, буду сохранять данные в словари.

In [12]:
dict_m = {}
dict_sc = {}

In [13]:
def preprocess_data(features,dict_m,dict_sc):
    matrix = np.random.randint(10000,size = (len(features.columns),len(features.columns)))
    while not is_matrix_inv(matrix):
        matrix = np.random.randint(10000,size = (4,4))
    learn_id = len(dict_m)+1
    dict_m[learn_id] = matrix
    new_features = features.values@matrix
    scaler = StandardScaler()
    scaler.fit(new_features)
    new_features = scaler.transform(new_features)
    dict_sc[learn_id] = scaler
    return (new_features, learn_id)

In [14]:
def postprocess_data(features, learn_id, dict_m, dict_sc):
    matrix = dict_m[learn_id]
    scaler = dict_sc[learn_id]
    new_features = scaler.transform(features.values@matrix)
    return(new_features)

**Обоснование**

В случае, если сделать метод postprocess_data доступным для вызова из скрипта, написанного дата-сайентистом, получится схема, когда специалист получает обезличенные данные и уникальный идентификатор, который необходимо статично зашить в написанный скрипт, который будет работать с боевыми данными. 
Таким образом, мы сможем гарантировать, что и обучающие и продовые данные обрабатываются идентичным образом, при этом каждый раз используется уникальная матрица и уникальный скэлер, что значительно усложняет процесс восстановления первоначалльных данных. 

## Проверка алгоритма

Проверим написанные методы в цикле. 

In [15]:
scores = []
for i in range (1,100):
    new_features, learn_id = preprocess_data(features_train,dict_m,dict_sc)
    new_features_valid = postprocess_data(features_valid, i,dict_m,dict_sc)
    model = LinearRegression()
    model.fit(new_features, target_train)
    results = model.predict(new_features_valid)
    score = r2_score(target_valid, results)
    scores.append(score)
(np.array(scores).mean()/r2_score(target_valid, results)*100)

99.99999999995194

## Выводы

Представленные для разработки алгоритма шифрования данные оказались качетсвенными, предобработка не потребовалась. 

Гипотеза об умножении признаков на обратимую матрицу с сохранением качества обучения модели оказалась верной, но с ограничениями, так как возможно появление дисбаланса классов, вследствие умножения на числа, отличющиеся от исходных на несколько порядков. Поэтому в итоговом алгоритме обработки данных используется умножение признаков на матрицу с последующим скалированием, что позволяет нивелировать дисбаланс. 